## TODO

- Usare stesso Target e feature di Interpolazione
- Window e Target Size fissi - Così dovrebbe bastare una fully connected layer
- Guardare come si allenano i transformers
- Creare account Wandb

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer, MinMaxScaler

In [2]:
class CustomDataset(Dataset):
    TARGET_MASK_ID = 1
    FEATURES_MASK_ID = 0
    DAY_IN_MINUTES = 287
    
    EXCLUDE_FEATURES = [
        "Failure", "Frequency", "Status", "Others",
        "Code", "name", "deviceid", "serial",
        "SB0", "Phase", "NV10P", "TotalEnergy_QuotControl", 
        #"INV01_TotalEnergy", "INV02_TotalEnergy", "INV03_TotalEnergy",
        #"cos",
        #"sin",
        "state", "SOL", "NA16", 'soil_moisture_7_to_28cm (m³/m³)',
    ]
    
    def __init__(self, dataset_path, window=6, target_size=1, start_date='2022-06-01', end_date='2023-04-01', verbose=False):
        self.df = pd.read_csv(dataset_path, delimiter = ";")
        self.df['timestamp'] = pd.to_datetime(self.df['timestamp'], yearfirst=True)
        cols_to_remove = [column  for column in self.df.columns if len([elem for elem in self.EXCLUDE_FEATURES if elem in column]) > 0]
        self.df = self.df.drop(columns=cols_to_remove)
        
        # filtro per le date contenute nel range start_date < x < end_date
        self.df = self.df[self.df['timestamp'] > pd.to_datetime('2022-06-01')]
        self.df = self.df[self.df['timestamp'] < pd.to_datetime('2023-04-01')]
        
        self.scaler = MinMaxScaler().fit(self.df.drop(columns=['timestamp']))

        self.df.loc[:, self.df.columns != 'timestamp'] = self.scaler.transform(self.df.loc[:, self.df.columns != 'timestamp'])

        self.window = window
        self.target_size = target_size # numero di giorni da interpolare
        self.shape = self.df.shape
        self.verbose = verbose
        
        if self.verbose:
            # controllo se le colonne hanno dei NaN
            print(self.df.isna().any())
        

    def __len__(self):
        temp= (self.df.shape[0]/self.DAY_IN_MINUTES)-(2*(self.window) + (self.target_size))-1
        #print("TEMP:", temp)
        #print("DF ROWS:", self.df.shape[0])
        #print("PRE DIFF", (2*(self.window*self.DAY_IN_MINUTES) + (self.target_size*self.DAY_IN_MINUTES)))
        #print((2*(self.window*self.DAY_IN_MINUTES) + (self.target_size*self.DAY_IN_MINUTES)))
        return int(temp)
        #return len(self.df)
    
    def __getitem__(self, idx):
        days = self.df.iloc[0+(idx*self.DAY_IN_MINUTES): self.DAY_IN_MINUTES*(self.window+self.target_size+self.window)+(idx*self.DAY_IN_MINUTES), ~self.df.columns.str.contains('^Unnamed')]
        
        if self.verbose:
            display("Target from dataframe")
            display(days)
                
        days = days[days.columns.drop(list(days.filter(regex='^.*_name.*$')))]
        days = days[days.columns.drop(list(days.filter(regex='^Unnamed')))]
        days['timestamp'] = pd.to_numeric(days['timestamp'])
        
        if self.verbose:
            display("Target after processing")
            display(days)
        
        target = days[self.DAY_IN_MINUTES*self.window:self.DAY_IN_MINUTES*(self.window+self.target_size)]
        target = target['Cont_TotalEnergy(kWh)']
        
        data_left = days[0:self.DAY_IN_MINUTES*self.window]
        data_right = days[self.DAY_IN_MINUTES*(self.window+self.target_size):]
        data = pd.concat([data_left, data_right])
        
        # mask = [self.FEATURES_MASK_ID]*(self.window*self.DAY_IN_MINUTES) + [self.TARGET_MASK_ID]*(self.target_size*self.DAY_IN_MINUTES) + [self.FEATURES_MASK_ID]*(self.window*self.DAY_IN_MINUTES)
        
        #if self.verbose:
        #    display("Mask")
        #    display(mask)
        
        tensor_target = torch.tensor(target.values)
        tensor_target = torch.reshape(tensor_target, (self.target_size*self.DAY_IN_MINUTES, 1))
        tensor_features = torch.tensor(data.values)
        tensor_features = tensor_features[None, :]
        return tensor_target, tensor_features
    
    def set_verbose(self, value: bool):
        self.verbose = value

In [3]:
train_dataset = CustomDataset("merged_annuale_meteo.csv", window=1, verbose=True)

timestamp                          False
INV01_CurrentDC(A)                 False
INV01_CurrentAC(A)                 False
INV01_TotalEnergy(kWh)             False
INV01_PowerAC(kW)                  False
INV01_PowerDC(kW)                  False
INV01_InternalTemperature(C)       False
INV01_HeatSinkTemperature(C)       False
INV01_VoltageDC(V)                 False
INV01_VoltageAC(V)                 False
INV02_CurrentDC(A)                 False
INV02_CurrentAC(A)                 False
INV02_TotalEnergy(kWh)             False
INV02_PowerAC(kW)                  False
INV02_PowerDC(kW)                  False
INV02_InternalTemperature(C)       False
INV02_HeatSinkTemperature(C)       False
INV02_VoltageDC(V)                 False
INV02_VoltageAC(V)                 False
INV03_CurrentDC(A)                 False
INV03_CurrentAC(A)                 False
INV03_TotalEnergy(kWh)             False
INV03_PowerAC(kW)                  False
INV03_PowerDC(kW)                  False
INV03_InternalTe

In [4]:
len(train_dataset)

300

In [5]:
train_dataset[102]

'Target from dataframe'

,timestamp,INV01_CurrentDC(A),INV01_CurrentAC(A),INV01_TotalEnergy(kWh),INV01_PowerAC(kW),INV01_PowerDC(kW),INV01_InternalTemperature(C),INV01_HeatSinkTemperature(C),INV01_VoltageDC(V),INV01_VoltageAC(V),...,month_sin,month_cos,hour_sin,hour_cos,temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),rain (mm),cloudcover (%),soil_temperature_7_to_28cm (°C)
63427,2022-09-11 00:05:00,0.0,0.0,0.444513,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.50000,1.000000,0.619938,0.724638,0.829653,0.0,0.01,0.845833
63428,2022-09-11 00:10:00,0.0,0.0,0.444513,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.50000,1.000000,0.619938,0.724638,0.829653,0.0,0.01,0.845833
63429,2022-09-11 00:15:00,0.0,0.0,0.444513,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.50000,1.000000,0.619938,0.724638,0.829653,0.0,0.01,0.845833
63430,2022-09-11 00:20:00,0.0,0.0,0.444513,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.50000,1.000000,0.619938,0.724638,0.829653,0.0,0.01,0.845833
63431,2022-09-11 00:25:00,0.0,0.0,0.444513,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.50000,1.000000,0.619938,0.724638,0.829653,0.0,0.01,0.845833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64283,2022-09-13 23:35:00,0.0,0.0,0.459241,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.37059,0.982963,0.498442,0.463768,0.586751,0.0,0.00,0.737500
64284,2022-09-13 23:40:00,0.0,0.0,0.459241,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.37059,0.982963,0.498442,0.463768,0.586751,0.0,0.00,0.737500
64285,2022-09-13 23:45:00,0.0,0.0,0.459241,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.37059,0.982963,0.498442,0.463768,0.586751,0.0,0.00,0.737500
64286,2022-09-13 23:50:00,0.0,0.0,0.459241,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.37059,0.982963,0.498442,0.463768,0.586751,0.0,0.00,0.737500


'Target after processing'

,timestamp,INV01_CurrentDC(A),INV01_CurrentAC(A),INV01_TotalEnergy(kWh),INV01_PowerAC(kW),INV01_PowerDC(kW),INV01_InternalTemperature(C),INV01_HeatSinkTemperature(C),INV01_VoltageDC(V),INV01_VoltageAC(V),...,month_sin,month_cos,hour_sin,hour_cos,temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),rain (mm),cloudcover (%),soil_temperature_7_to_28cm (°C)
63427,1662854700000000000,0.0,0.0,0.444513,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.50000,1.000000,0.619938,0.724638,0.829653,0.0,0.01,0.845833
63428,1662855000000000000,0.0,0.0,0.444513,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.50000,1.000000,0.619938,0.724638,0.829653,0.0,0.01,0.845833
63429,1662855300000000000,0.0,0.0,0.444513,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.50000,1.000000,0.619938,0.724638,0.829653,0.0,0.01,0.845833
63430,1662855600000000000,0.0,0.0,0.444513,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.50000,1.000000,0.619938,0.724638,0.829653,0.0,0.01,0.845833
63431,1662855900000000000,0.0,0.0,0.444513,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.50000,1.000000,0.619938,0.724638,0.829653,0.0,0.01,0.845833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64283,1663112100000000000,0.0,0.0,0.459241,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.37059,0.982963,0.498442,0.463768,0.586751,0.0,0.00,0.737500
64284,1663112400000000000,0.0,0.0,0.459241,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.37059,0.982963,0.498442,0.463768,0.586751,0.0,0.00,0.737500
64285,1663112700000000000,0.0,0.0,0.459241,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.37059,0.982963,0.498442,0.463768,0.586751,0.0,0.00,0.737500
64286,1663113000000000000,0.0,0.0,0.459241,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.37059,0.982963,0.498442,0.463768,0.586751,0.0,0.00,0.737500


(tensor([[0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.4569],
         [0.

In [6]:
train_dataset.set_verbose(False)

In [7]:
target, features = train_dataset[10]

print(target)
print(features)
print(target.shape, features.shape)

tensor([[0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0.0475],
        [0

In [8]:
sempler = SequentialSampler(train_dataset)

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=5, sampler=sempler, shuffle=False)

In [10]:
train_target, train_mask = next(iter(train_dataloader))

print("Len Target: ", len(train_target))
print("Len Features: ", len(train_mask))
print("Shape Target: ", train_target.shape)
print("Shape Features: ", train_mask.shape)
print()
print(train_target)
print(train_mask)

Len Target:  5
Len Features:  5
Shape Target:  torch.Size([5, 287, 1])
Shape Features:  torch.Size([5, 1, 574, 42])

tensor([[[0.0051],
         [0.0051],
         [0.0051],
         ...,
         [0.0102],
         [0.0102],
         [0.0102]],

        [[0.0102],
         [0.0102],
         [0.0102],
         ...,
         [0.0152],
         [0.0152],
         [0.0152]],

        [[0.0152],
         [0.0152],
         [0.0152],
         ...,
         [0.0195],
         [0.0195],
         [0.0195]],

        [[0.0195],
         [0.0195],
         [0.0195],
         ...,
         [0.0240],
         [0.0240],
         [0.0240]],

        [[0.0240],
         [0.0240],
         [0.0240],
         ...,
         [0.0289],
         [0.0289],
         [0.0289]]], dtype=torch.float64)
tensor([[[[1.6540e+18, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           1.0000e-01, 7.2917e-01],
          [1.6540e+18, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           1.0000e-01, 7.2917e-01],
          

In [11]:
# controllo il contenuto del primo elemento del primo Batch
for i in range(len(train_target[0])):
    print(pd.to_datetime(train_target[0][i][0]))
    

1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01 00:00:00.008807265
1970-01-01

In [12]:
a, b = train_dataset[301]
print(a.shape, b.shape)
print(pd.to_datetime(int(b[0][0][0])))
print(a, b)

torch.Size([287, 1]) torch.Size([1, 574, 42])
2023-03-29 00:05:00
tensor([[9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000],
        [9844713.0000]

In [13]:
for batch_idx, (target, features) in enumerate(train_dataloader):
    print(batch_idx, target.shape, features.shape)

#batch_index, (data, mask) = next(next(next(next(enumerate(train_dataloader)))))

0 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
1 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
2 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
3 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
4 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
5 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
6 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
7 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
8 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
9 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
10 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
11 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
12 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
13 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
14 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
15 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
16 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
17 torch.Size([5, 287, 1]) torch.Size([5, 1, 574, 42])
18 torch.Size([5, 28

In [14]:
print(data.shape)
print(mask.shape)

NameError: name 'data' is not defined

In [ ]:
a,b = train_dataset[29]
print(a.shape, b.shape)

In [ ]:
a,b = train_dataset[29]
print(a.shape, b.shape)

### Model

In [11]:
import torch.nn.functional as F
from torch import nn
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import wandb

In [12]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# device = "cpu"
print(f"Using {device} device")

Using cuda device


In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # convolution
        self.conv1 = nn.Conv2d(1, 20, 5, dtype=torch.float64)
        self.conv2 = nn.Conv2d(20, 10, 5, dtype=torch.float64)
        self.conv3 = nn.Conv2d(10, 5, 5, dtype=torch.float64)
        self.pool = nn.MaxPool2d(3, 2) # forse stride va tolto (?)
        self.flat = nn.Flatten()
        
        # fully connected
        self.linear1 = nn.Linear(24108, 12054, dtype=torch.float64)
        self.linear2 = nn.Linear(12054, 6027, dtype=torch.float64)
        self.linear3 = nn.Linear(6027, 3013, dtype=torch.float64)
        self.linear4 = nn.Linear(3013, 753, dtype=torch.float64)
    
        self.bnorm1 = nn.BatchNorm1d(12054, affine=True)
        self.bnorm2 = nn.BatchNorm1d(6027, affine=True)
        self.bnorm3 = nn.BatchNorm1d(3013, affine=True)
        self.bnorm4 = nn.BatchNorm1d(753, affine=True)
        self.droput = nn.Dropout()

        self.linearOut = nn.Linear(753, 287, dtype=torch.float64)
        self.reshape = nn.Unflatten(1, (287, 1))

    def forward(self, x):
        #print(x.shape)
        #x = F.relu(self.conv1(x))
        #x = F.relu(self.conv2(x))
        #x = self.pool(F.relu(self.conv3(x)))
        #x.view()
        x = self.flat(x)
        
        x = F.relu(self.bnorm1(self.linear1(x)))
        x = F.relu(self.bnorm2(self.linear2(x)))
        x = F.relu(self.bnorm3(self.linear3(x)))
        x = F.relu(self.bnorm4(self.linear4(x)))

        x = F.softplus(self.linearOut(x))

        x = self.reshape(x)
        #print(x.shape)
        #print(x)
        return x


In [15]:
model = NeuralNetwork().double().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(model)
print(optimizer)

NeuralNetwork(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(10, 5, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=24108, out_features=12054, bias=True)
  (linear2): Linear(in_features=12054, out_features=6027, bias=True)
  (linear3): Linear(in_features=6027, out_features=3013, bias=True)
  (linear4): Linear(in_features=3013, out_features=753, bias=True)
  (bnorm1): BatchNorm1d(12054, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bnorm2): BatchNorm1d(6027, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bnorm3): BatchNorm1d(3013, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bnorm4): BatchNorm1d(753, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (droput): Dropout(p=0.5,

In [16]:
num_epochs=100

In [17]:
model.train()
t_loss_list, v_loss_list = [], []

wandb.init(
    # set the wandb project where this run will be logged
    project="interpolation",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.001,
    "architecture": "UFC", # Ultra Fully Connected 🚀
    "dataset": "RofiloAnnuale",
    "epochs": num_epochs,
    }
)


for epoch in tqdm(range(num_epochs), unit="epoch"):    
    for batch_idx, (target, features) in enumerate(train_dataloader):
        target, features = target.to(device), features.to(device)
        
        train_probs = model(features)
        #print('Model resu: ', train_probs.shape)
        #print(features.shape, target.shape)
        #print(train_probs[0])
        #print(features)
        #print('Target : ', target.shape)

        #target = target[0:, None, None]
        #train_loss = F.mse_loss(target, train_probs)
        train_loss = F.l1_loss(target, train_probs)
        #train_loss = F.binary_cross_entropy(train_probs, labels.view(train_probs.shape))
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        # log metrics to wandb
        wandb.log({"loss": train_loss})
        
        t_loss_list.append(train_loss.item())
        
    print(
            f"Epoch {epoch+1:02d}/{num_epochs:02d}"
            f" | Train Loss {train_loss:.3f}"
        )

wandb.finish()
plt.plot(t_loss_list)

wandb: Currently logged in as: nicolo-vescera (rofilopd). Use `wandb login --relogin` to force relogin


  0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch 01/100 | Train Loss 0.524
Epoch 02/100 | Train Loss 0.198
Epoch 03/100 | Train Loss 0.216
Epoch 04/100 | Train Loss 0.213
Epoch 05/100 | Train Loss 0.183
Epoch 06/100 | Train Loss 0.194
Epoch 07/100 | Train Loss 0.293
Epoch 08/100 | Train Loss 0.244
Epoch 09/100 | Train Loss 0.243
Epoch 10/100 | Train Loss 0.236
Epoch 11/100 | Train Loss 0.284
Epoch 12/100 | Train Loss 0.231
Epoch 13/100 | Train Loss 0.221
Epoch 14/100 | Train Loss 0.260
Epoch 15/100 | Train Loss 0.221
Epoch 16/100 | Train Loss 0.266
Epoch 17/100 | Train Loss 0.241
Epoch 18/100 | Train Loss 0.248
Epoch 19/100 | Train Loss 0.231
Epoch 20/100 | Train Loss 0.343
Epoch 21/100 | Train Loss 0.364
Epoch 22/100 | Train Loss 0.360
Epoch 23/100 | Train Loss 0.384
Epoch 24/100 | Train Loss 0.387
Epoch 25/100 | Train Loss 0.375
Epoch 26/100 | Train Loss 0.379
Epoch 27/100 | Train Loss 0.383
Epoch 28/100 | Train Loss 0.399
Epoch 29/100 | Train Loss 0.398
Epoch 30/100 | Train Loss 0.392
Epoch 31/100 | Train Loss 0.402
Epoch 32

KeyboardInterrupt: 

In [18]:
wandb.finish()

loss,▃▂▂▂▁▁▁▂▇▅▄▃▂▃█▇▃▃▃▃▁▂▂▃▃▂▁▂▅▄▄▃▄▄▆▅▃▃▃▁
loss,0.21492


In [31]:
torch.cuda.empty_cache()
target = None
features = None
t_loss_list = None
train_probs = None
train_loss = None
model = None
torch.cuda.empty_cache()

In [30]:
import gc

model.cpu()
# del model, checkpoint
del model
gc.collect()
torch.cuda.empty_cache()

## WanDB

In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Currently logged in as: nicolo-vescera (rofilopd). Use `wandb login --relogin` to force relogin


acc,▁▂▅▅▇█▇█
loss,▆█▇▅▄▁▁▁
acc,0.79953
loss,0.19777


✔✓